# Grafos Ponderados

In [1]:
#include <iostream>
#include <string>
#include <set>
#include <queue>

using namespace std;

In [2]:
class Grafo {
    public:
        static const std::size_t MAXIMUM = 20;
    private:
        int arestas[MAXIMUM][MAXIMUM];
        int labels[MAXIMUM];
        std::size_t quantidade_vertices;
    public:
        Grafo();
        void adicionarVertice(int label);
        bool adicionarAresta(size_t origem, size_t destino, int peso);
        bool removerAresta(size_t origem, size_t destino);
        string listarVertices() const;
        int retornaVertice(std::size_t vertice) const;
        std::size_t quantidadeVertices() const;
        bool ehAresta(size_t origem, size_t destino) const;
        std::set<std::size_t> vizinhos(std::size_t vertice) const;
        void buscaProfundidade(int inicio) const;
        void buscaLargura(int inicio) const;
        void menoresDistancias(int origem) const;
    private:
        void auxBuscaProfundidade(int v, bool anotados[]) const;
        int minDistance(int dist[], bool sptSet[]) const;
};

In [3]:
Grafo::Grafo() {
    quantidade_vertices = 0;
    for (int i = 0 ; i < MAXIMUM ; i++) {
        for (int j = 0 ; j < MAXIMUM ; j ++) {
            arestas[i][j] = -1;
        }
    }
}

In [4]:
void Grafo::adicionarVertice(int label) {
    std::size_t novoVertice;
    labels[quantidade_vertices] = label;
    novoVertice = quantidade_vertices;
    quantidade_vertices++;
    
    for(int i = 0 ; i < quantidade_vertices ; i ++) {
        arestas[i][novoVertice] = -1;
        arestas[novoVertice][i] = -1;
    }
}

In [5]:
string Grafo::listarVertices() const {
    string ret;
    for (int i = 0 ; i < quantidade_vertices ; i++)
        ret = ret + "[" + std::to_string(labels[i]) + "]";
    return ret;
}

In [6]:
int Grafo::retornaVertice(std::size_t vertice) const {
    assert(vertice < quantidade_vertices);
    return labels[vertice];
}

In [7]:
std::size_t Grafo::quantidadeVertices() const {
    return quantidade_vertices;
}

In [8]:
bool Grafo::adicionarAresta(size_t origem, size_t destino, int peso) {
    if (origem >= quantidade_vertices || destino >= quantidade_vertices)
        return false;
    arestas[origem][destino] = peso;
    return true;
}

In [9]:
bool Grafo::removerAresta(size_t origem, size_t destino) {
    if (origem >= quantidade_vertices || destino >= quantidade_vertices)
        return false;
    arestas[origem][destino] = -1;
    return true;
}

In [10]:
bool Grafo::ehAresta(size_t origem, size_t destino) const {
    if (origem >= quantidade_vertices || destino >= quantidade_vertices)
        return false;
    return (arestas[origem][destino] > -1);
}    

In [11]:
std::set<std::size_t> Grafo::vizinhos(std::size_t vertice) const {
    std::set<std::size_t> ret;
    for (int i = 0 ; i < quantidadeVertices() ; i++) {
        if (arestas[vertice][i] > -1) {
            ret.insert(i);
        }
    }
    return ret;
}

### Depth-First Search

Na teoria dos grafos, busca em profundidade (ou busca em profundidade-primeiro, também conhecido em inglês por Depth-First Search - DFS) é um algoritmo usado para realizar uma busca ou travessia numa árvore, estrutura de árvore ou grafo. Intuitivamente, o algoritmo começa num nó raiz (selecionando algum nó como sendo o raiz, no caso de um grafo) e explora tanto quanto possível cada um dos seus ramos, antes de retroceder(backtracking).

In [12]:
void Grafo::auxBuscaProfundidade(int v, bool anotados[]) const {
    std::set<std::size_t> conexoes = vizinhos(v);
    anotados[v] = true;
    std::cout << labels[v] << std::endl;    
    for (auto item: conexoes) {
        if (! anotados[item]) {
            auxBuscaProfundidade(item, anotados);
        }
    }    
}

In [13]:
void Grafo::buscaProfundidade(int inicio) const {
    bool anotados[MAXIMUM];
    std::fill_n(anotados, quantidadeVertices(), false); // inicializando anotados
    auxBuscaProfundidade(inicio, anotados);
}

### Breadth-First Search

Na teoria dos grafos, busca em largura (ou busca em amplitude, também conhecido em inglês por Breadth-First Search - BFS) é um algoritmo de busca em grafos utilizado para realizar uma busca ou travessia num grafo e estrutura de dados do tipo árvore. Intuitivamente, você começa pelo vértice raiz e explora todos os vértices vizinhos. Então, para cada um desses vértices mais próximos, exploramos os seus vértices vizinhos inexplorados e assim por diante, até que ele encontre o alvo da busca.

In [14]:
void Grafo::buscaLargura(int inicio) const {
    bool anotados[MAXIMUM];
    std::set<std::size_t> conexoes;
    std::queue<std::size_t> filaVertices;
    
    std::fill_n(anotados, quantidadeVertices(), false); // inicializando anotados
    
    anotados[inicio] = true;
    std::cout << labels[inicio] << std::endl;
    filaVertices.push(inicio);
    do {
        conexoes = vizinhos(filaVertices.front());
        filaVertices.pop();
        for (auto item: conexoes) {
            if (! anotados[item]) {
                anotados[item] = true;
                std::cout << labels[item] << std::endl;
                filaVertices.push(item);
            }
        }
    } while(!filaVertices.empty());
}

In [15]:
// A utility function to find the vertex with minimum
// distance value, from the set of vertices not yet included
// in shortest path tree
int Grafo::minDistance(int dist[], bool sptSet[]) const {
    // Initialize min value
    int min = 99999, min_index;
    for (int v = 0; v < quantidadeVertices() ; v++)
        if (sptSet[v] == false && dist[v] <= min)
            min = dist[v], min_index = v;
    return min_index;
}

### Menores Distancias: Dijkstra

In [16]:
void Grafo::menoresDistancias(int origem) const {
    int distancias[MAXIMUM]; // vetor contendo as minimas distancias a partir da origem
    bool precedentes[MAXIMUM]; // vetor indicando se um vertice já foi processado
    for (int i = 0; i < MAXIMUM; i++) {
        distancias[i] = 99999;
        precedentes[i] = false;
    }
    std::cout << "init ok" << std::endl;
    distancias[origem] = 0; 
    for (int count = 0; count < quantidadeVertices() - 1; count++) {
        int u = minDistance(distancias, precedentes); // vertice com menor distancia entre os não processados 
        precedentes[u] = true; // marcar como processado
 
        // Update dist value of the adjacent vertices of the
        // picked vertex.
        for (int v = 0; v < quantidadeVertices(); v++) {
            // Update dist[v] only if is not in sptSet,
            // there is an edge from u to v, and total
            // weight of path from src to  v through u is
            // smaller than current value of dist[v]
            if (!precedentes[v] && arestas[u][v] != -1 && distancias[u] != 99999 && distancias[u] + arestas[u][v] < distancias[v]) {
                distancias[v] = distancias[u] + arestas[u][v];
            }
        }
    }
 
    for (int v = 0; v < quantidadeVertices(); v++) {
        if (distancias[v] != 99999) {
            std::cout << v << " - " << distancias[v] << std::endl;
        }
    }
}

In [17]:
//t2.quantidadeVertices()

In [18]:
Grafo t;

In [19]:
t.adicionarVertice(3);
cout << t.quantidadeVertices() << endl;
t.adicionarVertice(2);
cout << t.quantidadeVertices() << endl;

1
2


In [20]:
cout << t.listarVertices();

[3][2]

In [21]:
cout << t.retornaVertice(0) << "\n" << t.retornaVertice(1) << endl;

3
2


In [22]:
cout << t.retornaVertice(3) << "\n" << t.retornaVertice(1) << endl;

0
2


In [23]:
t.adicionarAresta(2,1,11)

false

In [24]:
t.ehAresta(0,1)

false

In [25]:
t.adicionarAresta(0,1,12)

true

In [26]:
t.ehAresta(0,1)

true

In [27]:
t.ehAresta(1,0)

false

In [28]:
t.vizinhos(0)

{ 1 }

In [29]:
t.vizinhos(1)

{}

In [30]:
t.adicionarVertice(2)

In [31]:
t.adicionarAresta(0,2,10)

true

In [32]:
t.vizinhos(0)

{ 1, 2 }

In [33]:
t.removerAresta(0,1)

true

In [34]:
t.vizinhos(0)

{ 2 }

In [35]:
t.ehAresta(0,1)

false

In [36]:
t.ehAresta(1,0)

false

In [37]:
t.adicionarAresta(0,1,20)

true

In [38]:
t.vizinhos(0)

{ 1, 2 }

In [39]:
t.vizinhos(1)

{}

In [40]:
t.vizinhos(2)

{}

### Testando Busca em Profundidade e Largura

In [41]:
Grafo t2;
for (int i = 0 ; i < 7 ; i++)
    t2.adicionarVertice(i);
cout << t2.listarVertices();

[0][1][2][3][4][5][6]

In [42]:
t2.adicionarAresta(0,1,10);
t2.adicionarAresta(0,4,10);

t2.adicionarAresta(1,3,10);

t2.adicionarAresta(2,0,10);

t2.adicionarAresta(3,0,10);
t2.adicionarAresta(3,5,10);
t2.adicionarAresta(3,6,10);

In [43]:
t2.vizinhos(0)

{ 1, 4 }

In [44]:
t2.vizinhos(1)

{ 3 }

In [45]:
t2.vizinhos(2)

{ 0 }

In [46]:
t2.vizinhos(3)

{ 0, 5, 6 }

In [47]:
t2.vizinhos(4)

{}

In [48]:
t2.vizinhos(5)

{}

In [49]:
t2.vizinhos(6)

{}

In [50]:
t2.buscaProfundidade(0);

0
1
3
5
6
4


In [51]:
t2.buscaLargura(0);

0
1
4
3
5
6


In [52]:
t2.menoresDistancias(0);

init ok
0 - 0
1 - 10
3 - 20
4 - 10
5 - 30
6 - 30


### Testando Dijkstra

In [54]:
Grafo t3;
for (int i = 0 ; i < 6 ; i++)
    t3.adicionarVertice(i);
cout << t3.listarVertices();

[0][1][2][3][4][5]

In [55]:
t3.adicionarAresta(0, 1, 2);
t3.adicionarAresta(0, 5, 9);

t3.adicionarAresta(1, 2, 8);
t3.adicionarAresta(1, 3, 15);
t3.adicionarAresta(1, 5, 6);

t3.adicionarAresta(2, 3, 1);

t3.adicionarAresta(4, 2, 7);
t3.adicionarAresta(4, 3, 3);

t3.adicionarAresta(5, 4, 3);

In [56]:
t3.menoresDistancias(0);

init ok
0 - 0
1 - 2
2 - 10
3 - 11
4 - 11
5 - 8
